In [1]:
%load_ext tensorboard

In [2]:
import datetime
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import keras

from tqdm import tqdm

2024-08-01 15:07:26.543568: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-01 15:07:26.673357: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-01 15:07:26.673405: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-01 15:07:26.691284: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-01 15:07:26.736647: I tensorflow/core/platform/cpu_feature_guar

In [3]:
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.models import load_model

In [4]:
# gpu_options = tf.GPUOptions(visible_device_list="0")
# sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


2024-08-01 15:07:28.515566: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-01 15:07:28.581582: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-01 15:07:28.581718: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [6]:
gpus = tf.config.list_physical_devices('GPU')

if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)


tf.debugging.set_log_device_placement(True)


1 Physical GPUs, 1 Logical GPU


2024-08-01 15:07:28.592384: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-01 15:07:28.592553: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-01 15:07:28.592622: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [7]:
# import cv2
# import urllib
# import requests
# import PIL.Image

# from bs4 import BeautifulSoup

In [8]:
# #downloading ship synset
# ship_page = requests.get("http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=n03095699")
# ship_soup = BeautifulSoup(ship_page.content, "html.parser")

# #downloading bike synset:
# bike_page = requests.get("http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=n03792782")
# bike_soup = BeautifulSoup(bike_page.content, "html.parser")

# ship_soup_str = str(ship_soup)
# ship_soup_split_url = ship_soup_str.split("\r\n")

# bike_soup_str = str(bike_soup)
# bike_soup_split_url = bike_soup_str.split("\r\n")

In [9]:
# os.mkdir('./content')
# os.mkdir('./content/train')
# os.mkdir('./content/train/ships')
# os.mkdir('./content/train/bikes')
# os.mkdir('./content/validation')
# os.mkdir('./content/validation/ships')
# os.mkdir('./content/validation/bikes')

In [10]:
# print("le: ", len(bike_soup_split_url))

In [11]:
# image_rows, image_cols = 32, 32

# input_shape = (image_rows, image_cols, 3)

# def url_to_image(url):
#     resp = urllib.request.urlopen(url)
#     image = np.asarray(bytearray(resp.read()), dtype="uint8")
#     image = cv2.imdecode(image, cv2.IMREAD_COLOR)
#     return image

# n_training_images = 100

# #train images for ship
# for progress in tqdm(range(n_training_images)):
#     if not ship_soup_split_url[progress] == None:
#         try:
#             I = url_to_image(ship_soup_split_url[progress])
#             if len(I.shape) == 3:
#                 save_path = "./content/train/ships/img" + str(progress) + ".jpeg"
#                 cv2.imwrite(save_path, I)
#         except:
#             None

# #train images for bikes
# for progress in tqdm(range(n_training_images)):
#     if not bike_soup_split_url[progress] == None:
#         try:
#             I = url_to_image(bike_soup_split_url[progress])
#             if len(I.shape) == 3:
#                 save_path = "./content/train/bikes/img" + str(progress) + ".jpeg"
#                 cv2.imwrite(save_path, I)
#         except:
#             None

# #validation images for ships
# for progress in tqdm(range(50)):
#     if not ship_soup_split_url[n_training_images+progress] == None:
#         try:
#             I = url_to_image(ship_soup_split_url[n_training_images+progress])
#             if len(I.shape) == 3:
#                 save_path = "./content/validation/ships/img" + str(progress) + ".jpeg"
#                 cv2.imwrite(save_path, I)
#         except:
#             None


# #validation images for bikes
# for progress in tqdm(range(50)):
#     if not bike_soup_split_url[n_training_images+progress] == None:
#         try:
#             I = url_to_image(bike_soup_split_url[n_training_images+progress])
#             if len(I.shape) == 3:
#                 save_path = "./content/validation/bikes/img" + str(progress) + ".jpeg"
#                 cv2.imwrite(save_path, I)
#         except:
#             None

In [12]:
#defining model

class AlexNet(Sequential):
    def __init__(self, input_shape, num_classes):
        super().__init__()

        self.add(Conv2D(
            96, 
            kernel_size=(11, 11), 
            strides=4, 
            padding="valid", 
            activation="relu", 
            input_shape=input_shape,
            kernel_initializer="he_normal"
        ))
        
        self.add(MaxPooling2D(
            pool_size=(3, 3),
            strides = (2, 2),
            padding = "valid",
            data_format = None
        ))

        self.add(Conv2D(
            256,
            kernel_size=(5, 5),
            strides=1,
            padding="same",
            activation="relu",
            kernel_initializer="he_normal"
        ))

        self.add(MaxPooling2D(
            pool_size=(3, 3),
            strides = (2, 2),
            padding="valid",
            data_format=None
        ))

        self.add(Conv2D(
            384,
            kernel_size=(3, 3),
            strides=1,
            padding="same",
            activation="relu",
            kernel_initializer="he_normal"
        ))

        self.add(Conv2D(
            384,
            kernel_size=(3, 3),
            strides=1,
            padding="same",
            activation="relu",
            kernel_initializer="he_normal"
        ))

        self.add(Conv2D(
            256,
            kernel_size=(3, 3),
            strides=1,
            padding="same",
            activation="relu",
            kernel_initializer="he_normal"
        ))

        self.add(MaxPooling2D(
            pool_size=(3, 3),
            strides=(2, 2),
            padding="valid",
            data_format=None
        ))

        self.add(Dropout(0.5))
        self.add(Flatten())
        self.add(Dropout(0.5))
        self.add(Dense(4096, activation="relu"))
        self.add(Dense(4096, activation="relu"))
        self.add(Dense(1000, activation="relu"))
        self.add(Dense(num_classes, activation="softmax"))

        self.compile(
            optimizer=Adam(0.0001),
            loss = 'categorical_crossentropy',
            metrics=["accuracy"]
        )


In [13]:
num_classes = 3

model = AlexNet((227, 227, 3), num_classes)

epochs = 1
batch_size = 32

train_dir = "/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Imagenet/synsets"
valid_dir = "/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Imagenet/valid_synsets"

Image_height = 227
Image_width = 227

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (Image_height, Image_width),
    color_mode="rgb",
    batch_size=batch_size,
    seed=1,
    shuffle=True,
    class_mode="categorical"
)

valid_datagen = ImageDataGenerator(
    rescale=1./255
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(Image_height, Image_width),
    color_mode="rgb",
    batch_size=batch_size,
    seed=7,
    shuffle=True,
    class_mode="categorical"
)

train_num = train_generator.samples
valid_num = valid_generator.samples

Found 3605 images belonging to 3 classes.
Found 264 images belonging to 3 classes.


In [14]:
# Has to be executed only once
# os.mkdir("./logs")
# os.mkdir("./logs/fit")

In [15]:
log_dir = "./logs/fit" + datetime.datetime.now().strftime("%d%m%Y-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
callbacks_list = [tensorboard_callback]

model.fit(
    train_generator,
    epochs=epochs,
    steps_per_epoch=train_num//batch_size,
    validation_data=valid_generator,
    validation_steps=valid_num//batch_size,
    callbacks=callbacks_list,
    verbose=1
)

model.summary()

2024-08-01 15:07:29.289770: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-08-01 15:07:30.003742: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inalex_net/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-08-01 15:07:30.416077: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-08-01 15:07:30.526545: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-08-01 15:07:32.052604: I external/local_xla/xla/service/service.cc:168] XLA service 0x7bbc40fc5130 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-01 15:07:32.052630: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050

112/112 [==============================] - 35s 237ms/step - loss: 0.8631 - accuracy: 0.6566 - val_loss: 0.4522 - val_accuracy: 0.8633
Model: "alex_net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 55, 55, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2  (None, 27, 27, 96)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 256)       614656    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 13, 13, 256)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 13, 13, 384)       8

In [16]:
model.summary()

Model: "alex_net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 55, 55, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2  (None, 27, 27, 96)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 256)       614656    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 13, 13, 256)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 13, 13, 384)       885120    
                                                                 
 conv2d_3 (Conv2D)           (None, 13, 13, 384)       132

In [28]:
model_dir = "/home/abhinav/Documents/Work/2 Hobby_projects/Models/myalexnet.h5"

In [29]:
# Clear all previously registered custom objects
keras.saving.get_custom_objects().clear()


NameError: name 'printf' is not defined

In [30]:
model.save(model_dir)

/home/abhinav/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [31]:
n = load_model(model_dir)

ValueError: Unknown layer: 'AlexNet'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [ ]:
new_model_pre = keras.models.load_model("/home/abhinav/Documents/Work/2 Hobby_projects/Models/myalexnet.h5")

ValueError: Unknown layer: 'AlexNet'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [23]:
#predictions

class_names = ['chipmunk', 'mountain_bike', 'ship']

x_valid, label_batch = next(iter(valid_generator))
# print("x_valid: ", x_valid)
# print("label_batch: ", label_batch)

print("x_valid_shape: ", x_valid.shape)

predict_x = n.predict(x_valid)
print("predict_x: ", predict_x)
prediction_values = np.argmax(predict_x,axis=1)
print("prediction_values", prediction_values)

fig = plt.figure(figsize=(10, 6))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

for i in range(8):
    ax = fig.add_subplot(2, 4, i+1, xticks=[], yticks=[])
    ax.imshow(x_valid[i, :], cmap=plt.cm.gray_r, interpolation='nearest')
    if prediction_values[i] == np.argmax(label_batch[i]):
        print("label_batch[i]: ", label_batch[i])
        print("prediction_values[i]:  is the same as np.argmax(label_batch[i])", prediction_values[i])
        print("class_names: ", class_names)
        ax.text(3, 17, class_names[prediction_values[i]], color='blue', fontsize=14)
    else:
        ax.text(3, 17, class_names[prediction_values[i]], color='red', fontsize=14)

x_valid_shape:  (32, 227, 227, 3)


AttributeError: 'CheckpointLoadStatus' object has no attribute 'predict'